In [ ]:
!pip install opendatasets

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd gdrive
%cd MyDrive
%cd transfer-learning-yolo-logos
!ls

In [ ]:
!unzip -q "/content/gdrive/MyDrive/transfer-learning-yolo-logos/darknet/build/darknet/x64/data/obj.zip" -d "/content/gdrive/MyDrive/transfer-learning-yolo-logos/darknet/build/darknet/x64/data"

In [ ]:
%cd transfer-learning-yolo-logos

In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git

In [ ]:
import opendatasets as od

dataset_url = 'https://www.kaggle.com/lyly99/logodet3k'
od.download(dataset_url)

In [ ]:
import os

root = 'logodet3k/LogoDet-3K'

In [ ]:
with open('darknet/build/darknet/x64/data/obj.names', mode='w+') as f:
  f.write('Logo')

In [ ]:
for d in os.listdir(root):
  for logo in os.listdir(os.path.join(root, d)):
    for file in os.listdir(os.path.join(root, d, logo)):
      os.rename(os.path.join(root, d, logo, file), os.path.join(root, d, logo) + '/' + logo + '_' + file)

In [ ]:
import shutil
from tqdm import tqdm

print(os.path.exists('darknet/build/darknet/x64/data/obj'))
if not os.path.exists('darknet/build/darknet/x64/data/obj'):
  os.makedirs('darknet/build/darknet/x64/data/obj')

for r, d, f in os.walk(root):
  for file in tqdm(f):
      if file.endswith('.jpg'):
          shutil.copyfile(os.path.join(r, file), 'darknet/build/darknet/x64/data/obj/' + file)

In [ ]:
import xml.etree.ElementTree as ET
import os

root = 'logodet3k/LogoDet-3K'
done = 0

for d in os.listdir(root):
  for logo in os.listdir(os.path.join(root, d)):
    for file in os.listdir(os.path.join(root, d, logo)):
        if file.endswith('.xml'):
          tree = ET.parse(os.path.join(root, d, logo, file))
          tree_root = tree.getroot()
          tags = [(i.tag, i.text) for i in tree_root.iter()]
          box_info = []
          for i, tag in enumerate(tags):
            if tag[0] == 'xmin':
              box_info.append(tags[i:i+4])

            if tag[0] == 'width':
              width = int(tag[1])

            if tag[0] == 'height':
              height = int(tag[1])
          
          open('darknet/build/darknet/x64/data/obj/' + file[:-4] + '.txt', 'w').close()

          with open('darknet/build/darknet/x64/data/obj/' + file[:-4] + '.txt', 'w+') as txt:
            for box in box_info:
              xmin = int(box[0][1])
              ymin = int(box[1][1])
              xmax = int(box[2][1])
              ymax = int(box[3][1])

              box_width_relative = (xmax - xmin) / width
              box_height_relative = (ymax - ymin) / height
              box_center_x_relative = ((xmax + xmin) / 2) / width
              box_center_y_relative = ((ymax + ymin) / 2) / height

              txt.write(' '.join(['0', str(box_center_x_relative), str(box_center_y_relative), str(box_width_relative), str(box_height_relative)]))
              txt.write('\n')
          
          done += 1
          print('done {} logos'.format(done))
                            

In [ ]:
from sklearn.model_selection import train_test_split
import random
import os
root = 'logodet3k/LogoDet-3K'

train = open('darknet/build/darknet/x64/data/train.txt', 'w+')
test = open('darknet/build/darknet/x64/data/test.txt', 'w+')
val = open('darknet/build/darknet/x64/data/val.txt', 'w+')
total_set = []
'''
for r, d, f in os.walk(root):
    for file in f:
        if file.endswith('.jpg'):
            train.write('build/darknet/x64/data/obj/' + file + '\n')
'''

for r, d, f in os.walk(root):
  for file in f:
      if file.endswith('.jpg'):
        total_set.append(file)

for file in total_set:
  train.write('build/darknet/x64/data/obj/' + file + '\n')
  

In [ ]:
%cd darknet

In [ ]:
!ls

In [ ]:
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!make

In [ ]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

In [ ]:
!./darknet detector train build/darknet/x64/data/obj.data cfg/yolo-obj.cfg build/darknet/x64/yolov4.conv.137 -dont_show

In [ ]:
image_name = "mickeys.jpg"
!./darknet detector test build/darknet/x64/data/obj.data cfg/yolo-obj.cfg backup/yolo-obj_last.weights image_name -dont_show

In [ ]:
imShow('predictions.jpg')